<a href="https://colab.research.google.com/github/sakshamtawakley/CC-Task/blob/main/cc_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas
import re
with open("timetable.log", "r") as f:
        log_data = f.readlines()


In [16]:
parsed_data=[]
total_api=0
end_point_section=0
end_point_courses=0
sum_endpoint_courses=0
sum_endpoint_section=0
max_endpoint_courses=0
max_endpoint_section=0
status_codes_200=0
status_codes_403=0
status_codes_404=0
heuristic=0
iterative=0
num_timetables=0
num_timetables_gen=0

In [17]:
pattern_user = re.compile( #pattern for messages having user id and endpoints
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] router: (?P<path>\S+)(?: \[(?P<user_id>[^\]]+)\])?"
)

pattern_http = re.compile( #pattern for messages having endpoints and duration of generation
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] (?P<method>\S+) (?P<path>\S+) "
    r"(?P<status>\d+) (?P<duration>[\d\.]+(?:µs|ms))"
)

pattern_msg = re.compile( #pattern for other messages
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] --- (?P<message>.+) ---"
)

In [18]:
def string_to_time (s): #function for replacing the time duration from string to a float type
                if s.endswith("µs"):
                  return float(s.replace("µs", "")) * 1e-6
                elif s.endswith("ms"):
                  return float(s.replace("ms", "")) * 1e-3
                elif s.endswith("s"):
                 return float(s.replace("s", ""))

In [19]:
id_2022=[]
id_2023=[]
id_2024=[]
id_2025=[]
#2024A*PS0660P
def search(user_id): # segregating id no.s based on roll numbers
  if user_id is None:
    return
  if user_id[:4]=="2022":
    for i in id_2022:
      if user_id[8:12]==i[8:12]:
        return
    id_2022.append(user_id)
    return
  elif user_id[:4]=="2023":
    for i in id_2023:
      if user_id[8:12]==i[8:12]:
        return
    id_2023.append(user_id)
    return
  elif user_id[:4]=="2024":
    for i in id_2024:
      if user_id[8:12]==i[8:12]:
        return
    id_2024.append(user_id)
    return
  elif user_id[:4]=="2025":
    for i in id_2025:
      if user_id[8:12]==i[8:12]:
        return
    id_2025.append(user_id)
    return


In [20]:
for i in log_data: #parsing our file
  i=i.strip()
  m1 = pattern_http.match(i)
  m2 = pattern_user.match(i)
  m3 = pattern_msg.match(i)
  if m1:
            total_api=total_api+1
            val=string_to_time(m1["duration"])
            if m1["path"] == "/courses": #check for /courses endpoint
                end_point_courses=end_point_courses+1
                if val is not None:
                  sum_endpoint_courses+=val
                  max_endpoint_courses=max(max_endpoint_courses, val)
            elif m1["path"] == "/sections": #check for /sections endpoint
                end_point_section=end_point_section+1
                if val is not None:
                  sum_endpoint_section+=val
                  max_endpoint_section=max(max_endpoint_section, val)

            if int(m1["status"])==200: #check for status codes
                status_codes_200+=1
            elif int(m1["status"])==403:
                status_codes_403+=1
            elif int(m1["status"])==404:
                status_codes_404+=1
            parsed_data.append({ #appending our data into the appropriate data forms
                "date": m1["date"],
                "time": val,
                "ip": m1["ip"],
                "method": m1["method"],
                "path": m1["path"],
                "status": int(m1["status"]),
                "duration": m1["duration"],
                "user_id": None,
                "message": None
            })
  elif m2:
            total_api=total_api+1
            val=string_to_time(m2["time"])
            if m2["path"] == "/courses":
                end_point_courses+=1
                if val is not None:
                  sum_endpoint_courses+=val
                  max_endpoint_courses=max(max_endpoint_courses, val)
            elif m2["path"] == "/sections":
                end_point_section=end_point_section+1
                if val is not None:
                  sum_endpoint_section+=val
                  max_endpoint_sectiom=max(max_endpoint_sectiom, val)
            search(m2["user_id"])
            parsed_data.append({
                "date": m2["date"],
                "time": val,
                "ip": m2["ip"],
                "method": None,
                "path": m2["path"],
                "status": None,
                "duration": None,
                "user_id": m2["user_id"],
                "message": None
            })
  elif m3:
            if (m3["message"][:7]=="Using H"): #Using Heuristic timetable generation. To save time only Using H is used and same for iterative
              heuristic+=1
            if (m3["message"][:7]=="Using I"): #Using iterative timetable generation
              iterative+=1
            if (m3["message"][0]=="G"): #Message for generating timetable
             L=m3["message"].split()
             num_timetables+=1
             num_timetables_gen+=int(L[3])
            parsed_data.append({
                "date": m3["date"],
                "time": m3["time"],
                "ip": m3["ip"],
                "method": None,
                "path": None,
                "status": None,
                "duration": None,
                "user_id": None,
                "message": m3["message"]
            })
  else:
         #other lines
            parsed_data.append({
                "date": None, "time": None, "ip": None,
                "method": None, "path": None, "status": None,
                "duration": None, "user_id": None, "message": i
            })

In [21]:
print("The no. of total api calls is: ",total_api)
print("End point /section: ",end_point_section)
print("End point /courses: ",end_point_courses)
print("Having status code 200: ",status_codes_200)
print("Having status code 403: ",status_codes_403)
print("Having status code 404: ",status_codes_404)
print("Heuristic time tables generated: ",heuristic)
print("Iterative time tables generated: ",iterative)
print("Average time taken for /section endpoints: ",sum_endpoint_section/end_point_section)
print("Average time taken for /courses endpoints: ",sum_endpoint_courses/end_point_courses)
print("Max time taken for /courses: ",max_endpoint_courses)
print("Max time taken for /sections: ",max_endpoint_section)
print("Number of timetables generated: ",num_timetables_gen)
print("Avg. Number of timetables generated: ",num_timetables_gen/num_timetables)
print("Number of students in 2022: ",len(id_2022))
print("Number of students in 2023: ",len(id_2023))
print("Number of students in 2024: ",len(id_2024))
print("Number of students in 2025: ",len(id_2025))


The no. of total api calls is:  145700
End point /section:  52090
End point /courses:  78004
Having status code 200:  60810
Having status code 403:  480
Having status code 404:  5
Heuristic time tables generated:  3806
Iterative time tables generated:  406
Average time taken for /section endpoints:  6.275128732962161e-05
Average time taken for /courses endpoints:  0.00018628487395518266
Max time taken for /courses:  0.009135005
Max time taken for /sections:  0.004969586
Number of timetables generated:  1705367
Avg. Number of timetables generated:  404.8829534662868
Number of students in 2022:  201
Number of students in 2023:  263
Number of students in 2024:  538
Number of students in 2025:  339
